In [1]:
import pandas as pd
from openai import OpenAI
from sklearn.metrics import matthews_corrcoef
from tqdm import tqdm
import torch
from transformers import pipeline, AutoTokenizer

In [6]:
motn = pd.read_csv('data/motn_test_text.csv')

# Llama 3 8B

In [3]:
model = "meta-llama/Meta-Llama-3-8B-Instruct"
pipe = pipeline("text-generation", model=model, torch_dtype=torch.bfloat16, device_map="auto")

C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


No Logit Bias

In [8]:
#system_message =  """You are a text classifier and are only allowed to respond with a 1 or a 0."""
user_message = """You are a classifier that determines what category a survey response belongs to. This survey asked people to respond to the question: "What does democracy mean to you?" Each answer belongs to one of the following 12 categories:

1. Freedom and Rights
2. One United Nation
3. Representation of the People
4. Popular Will and Equality
5. National Identity and Heritage
6. Not a Democracy, a Republic
7. Flawed Democracy
8. Institution and Constitution
9. Disaffected
10. I don't know
11. Nothing
12. Other

I will show you an answer to the question. Read the answer and then determine which of the categories most closely represents the respondent's answer. Return the number of the most appropriate category. Do not explain your answer, and only return the number. Here is the respondent's answer:
{}

Which category number best describes this answer?
"""

In [9]:
%%time
res = []
for doc in motn['comment_text']:
    messages = [
        #{
            #"role": "system",
            #"content": system_message,
        #},
        {"role": "user", "content": user_message.format(doc)},
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=2, do_sample=False, return_full_text = False, pad_token_id=pipe.tokenizer.eos_token_id, temperature = 0)
    res.extend(outputs)

res = [text['generated_text'] for text in res]

# return a list of unique responses from the model
print(set(res))

# add results to the dataframe
#motn['llama'] = [1 if '1' in text else 0 for text in res]
# export results
#motn.to_csv('./motn_test_llama3.csv', index = False)

C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\transformers\generation\configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\transformers\generation\configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'4', '7', '1', '6', '2', 'I cannot', '5', '10', '9', '11', '8', '3'}
CPU times: total: 1min 47s
Wall time: 1min 56s


In [13]:
motn['llama'] = res

In [15]:
motn.to_csv('llama_test.csv')